In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set, target table from BnF colbert 60

#Access and recess equation form the Parisian Alfonsines Tables
Equation_8sphere=HTable.read(238,format='dishas')
Hist_table=HTable.read(1160)
Hist_table

Years,Entries
yr,deg
IntegerAndSexagesimal,Historical
482130 ;,"08s 11 ; 22,42"
489435 ;,"08s 11 ; 35,13"
496740 ;,"08s 11 ; 47,35"
504045 ;,"08s 11 ; 59,48"
511350 ;,"08s 12 ; 11,52"
518655 ;,"08s 12 ; 23,45"
525960 ;,"08s 12 ; 36,27"
533265 ;,"08s 12 ; 46,28"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(11):
    Test_recomp[i][1]=Historical(Sexagesimal('0,0;0,0'),2)*u.degree
    Delta_recomp[i][1]=Historical(Sexagesimal('0,0;0,0'),2)*u.degree



In [4]:
REVO=Sexagesimal("6,0;0,0")*u.degree

# parameters from the Parisian Alfonsine tables
radix_apogee=Sexagesimal("3,53;23,42,4")*u.degree
radix_8sphere=Sexagesimal("5,59;12,34")*u.degree
daily_mean_8sphere=Sexagesimal("0;0,0,30,24,29")*u.degree
daily_mean_fixstars=Sexagesimal("0;0,0,4,20,41,17,12")*u.degree


for i in range(11):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=3):
# computation of the initial mean values     
        days=int(Hist_table[i][0])
        mean_8sphere=(radix_8sphere+days*daily_mean_8sphere)%REVO
        mean_fixstars=days*daily_mean_fixstars
#Equation of the eight sphere        
        eq_8sphere=Equation_8sphere.get(mean_8sphere)
#position of the apogee        
        apogee=(radix_apogee+mean_fixstars+eq_8sphere)%REVO
        apogee=Historical(apogee.value,2)
    with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
        Test_recomp[i][1]=1*apogee
        Delta_recomp[i][1]=Hist_table[i][1]-Test_recomp[i][1]

Delta_recomp

Years,Entries
yr,deg
IntegerAndSexagesimal,Historical
482130 ;,"00 ; 00,04"
489435 ;,"00 ; 00,03"
496740 ;,"00 ; 00,03"
504045 ;,"00 ; 00,03"
511350 ;,"00 ; 00,03"
518655 ;,"00 ; 00,03"
525960 ;,"00 ; 01,03"
533265 ;,"-00 ; 00,28"


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Saturn apogee.csv')
B.to_csv('Saturn apogee.zip', index=False, compression=compression_opts)